In [1]:
# Load Packages
from __future__ import unicode_literals, print_function

import plac #  wrapper over argparse
import random
from pathlib import Path
import spacy
from tqdm import tqdm, tqdm_notebook # loading bar 
import pandas as pd
import re
from pprint import pprint
from nltk.tokenize import sent_tokenize
import numpy as np

from spacy.util import minibatch, compounding

# Abrindo e gerando Datasets

### Entidade logradouro

In [2]:
# Obtendo dados crus de endereço

dataset = pd.read_csv("201906AGENCIAS.CSV")
dset = dataset.iloc[:-2,4:10].values  # tipo nparray object
pd_dset = pd.DataFrame(dset) # caso queira visualizar

pd_dset['Join'] = pd_dset[pd_dset.columns[0:]].apply(
    lambda x: ';'.join(x.dropna().astype(str)),
    axis=1
)
pd_dset.head(5)

,0,1,2,3,4,5,Join
0,"R.GUILHERME MOREIRA,315","SUBLOJA,LOJA E 2.ANDAR ...",CENTRO,69005-300,MANAUS ...,AM,"R.GUILHERME MOREIRA,315 ;SUBLO..."
1,"AV.PRES.VARGAS,248",1.E 2.ANDARES ...,CAMPINA,66010-900,BELEM ...,PA,"AV.PRES.VARGAS,248 ;1.E 2..."
2,"R.QUINZE DE NOVEMBRO,195",...,CENTRO,11010-908,SANTOS ...,SP,"R.QUINZE DE NOVEMBRO,195 ; ..."
3,"PCA.DAS QUATRO JORNADAS,11",MEZANINO ...,CENTRO,28010-000,CAMPOS DOS GOYTACAZES ...,RJ,"PCA.DAS QUATRO JORNADAS,11 ;MEZAN..."
4,"SEXTA AVENIDA,600",SECRETARIA DA EDUCACAO-TERREO ...,CAB,41745-002,SALVADOR ...,BA,"SEXTA AVENIDA,600 ;SECRE..."


In [3]:
# Tratando endereço completo do DataFrame
dset = np.array(pd_dset)
end_lista = []

for i in range(len(dset)):
    str_raw = dset[i][6]
    str_tratada = re.sub(r'[ ]{2,}', "",str_raw) # Tirando espaços excedentes no final do endereço
    str_tratada = re.sub(r'[;]{1,}', "; ", str_tratada) # Para complementos vazios, para não ter 2 ";"
    str_tratada = str_tratada.lower()
    
    end_lista.append(str_tratada)

end_lista[:3]

['r.guilherme moreira,315; subloja,loja e 2.andar; centro; 69005-300; manaus; am',
 'av.pres.vargas,248; 1.e 2.andares; campina; 66010-900; belem; pa',
 'r.quinze de novembro,195; centro; 11010-908; santos; sp']

In [4]:
# Lógica para extrair posição do logradouro:
# Dividir a String inteira por ";", pegar o len do primeiro split
# len de LOGRA é de 0 até len do primeiro split

print(end_lista[0])
split = end_lista[0].split(";")
print(split)
print(split[0])
print(len(split[0]))

r.guilherme moreira,315; subloja,loja e 2.andar; centro; 69005-300; manaus; am
['r.guilherme moreira,315', ' subloja,loja e 2.andar', ' centro', ' 69005-300', ' manaus', ' am']
r.guilherme moreira,315
23


In [5]:
# Extraindo iob com entidade apenas de logradouro

iob = []

for i in range(len(end_lista)):
    split = end_lista[i].split(";")
    len_logra = len(split[0])
    iob_dict = {"entities": [(0, len_logra-1,'LOGRA')]}
    end_lista[i] = end_lista[i].replace(";", " ")
    end_lista[i] = end_lista[i].replace(",", " ")
    end_lista[i] = re.sub(r'[ ]{2,}', " ", end_lista[i])

    
    tupla = (end_lista[i], iob_dict)
    iob.append(tupla)

FULL_DATA = iob

In [6]:
print(FULL_DATA[0])
print(FULL_DATA[1])
print(FULL_DATA[2])
print(FULL_DATA[5:15])

('r.guilherme moreira 315 subloja loja e 2.andar centro 69005-300 manaus am', {'entities': [(0, 22, 'LOGRA')]})
('av.pres.vargas 248 1.e 2.andares campina 66010-900 belem pa', {'entities': [(0, 17, 'LOGRA')]})
('r.quinze de novembro 195 centro 11010-908 santos sp', {'entities': [(0, 23, 'LOGRA')]})
[('av.rio branco 240 1.andar recife antigo 50030-310 recife pe', {'entities': [(0, 16, 'LOGRA')]}), ('av.santos dumont 2828 5.andar aldeota 60150-162 fortaleza ce', {'entities': [(0, 20, 'LOGRA')]}), ('pca.tiradentes 410 1.andar centro 80020-100 curitiba pr', {'entities': [(0, 17, 'LOGRA')]}), ('r.uruguai 185 5.andar centro 90010-901 porto alegre rs', {'entities': [(0, 12, 'LOGRA')]}), ('pca.1817 129 1.andar centro 58013-010 joao pessoa pb', {'entities': [(0, 11, 'LOGRA')]}), ('pca.odilon resende andrade 76 centro 37410-000 tres coracoes mg', {'entities': [(0, 28, 'LOGRA')]}), ('av fernandes lima 2591 terreo farol 57057-972 maceio al', {'entities': [(0, 22, 'LOGRA')]}), ('r.treze de junho 91

In [7]:
# Criação da base de teste e treinamento

n_test= 0.1 # Porcentagem para base de teste
test_n = round(len(FULL_DATA) * n_test)

# Divisao em Train Test Val

def gerador_bases(dataset, n):
    indices_random = random.sample(range(0,len(dataset)-1), n)
    base_teste_n = []
    base_treinamento_n = []
    
    for i in range(n):
        base_teste_n.append(dataset[indices_random[i]])

    for j in range(len(dataset)):
        if(j not in indices_random):
            base_treinamento_n.append(dataset[j])
            
    return base_teste_n, base_treinamento_n


base_teste, base_treinamento = gerador_bases(FULL_DATA, test_n)

random.shuffle(base_treinamento)
random.shuffle(base_teste)

print("Treinamento: " + str(len(base_treinamento)), "\nTeste: " + str(len(base_teste)), "\nTotal: " + str(len(FULL_DATA)))

Treinamento: 19202 
Teste: 2134 
Total: 21336


In [8]:
for i in range(5):
    print(base_treinamento[i])

('avenida �lvaro guimaraes 493 planalto 09890-002 sao bernardo do campo sp', {'entities': [(0, 28, 'LOGRA')]})
('rua 1. de maio 161 centro 85875-000 santa terezinha de itaipu pr', {'entities': [(0, 18, 'LOGRA')]})
('rua senador melo viana 158 centro 35740-000 esmeraldas mg', {'entities': [(0, 26, 'LOGRA')]})
('r.da conceicao 1270 centro 55540-000 palmares pe', {'entities': [(0, 18, 'LOGRA')]})
('r.jurubatuba 122 centro 09725-620 sao bernardo do campo sp', {'entities': [(0, 15, 'LOGRA')]})


# Carregando o modelo

In [13]:
# Define our variables

model = None
output_dir=Path(".")
n_iter= 100 # número de épocas
batch_size = 128

In [10]:
# Setting up the pipeline and entity recognizer.
if model is not None:
    nlp = spacy.load(model)  # load existing spacy model
    print("Loaded model '%s'" % model)
else:
    lan = 'pt'
    nlp = spacy.blank(lan)  # create blank Language class
    print("Created blank '%s' model" % lan)
    
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner)
    print('Added new NER')
else:
    ner = nlp.get_pipe('ner')
    print('Got an old NER')

Created blank 'pt' model
Added new NER


In [11]:
# create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
    ner = nlp.get_pipe('ner')

## Treinamento

In [14]:
print("Batch size: ", batch_size)
print("Épocas: ", n_iter)
print()

# add labels
for _, annotations in base_treinamento:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])


# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(base_treinamento)
        losses = {}
        batches = minibatch(base_treinamento, size=batch_size)
        
        for batch in batches:
            texts, annotations = zip(*batch)
            try:
                nlp.update(texts, annotations, sgd=optimizer, drop=0.2,
                       losses=losses)
            except:
                pass     
        
        print(itn+1, ' Losses', losses)        

Batch size:  128
Épocas:  100

1  Losses {'ner': 606.939372012857}
2  Losses {'ner': 638.5511046016755}
3  Losses {'ner': 766.4913897201557}
4  Losses {'ner': 609.2352539531776}
5  Losses {'ner': 559.8214193022287}
6  Losses {'ner': 588.6410193874871}
7  Losses {'ner': 547.6508409419387}
8  Losses {'ner': 455.7239492321868}
9  Losses {'ner': 413.08838508261584}
10  Losses {'ner': 601.2567512689601}
11  Losses {'ner': 730.5993479874415}
12  Losses {'ner': 643.234945708369}
13  Losses {'ner': 649.6897828269329}
14  Losses {'ner': 442.4402852816947}
15  Losses {'ner': 565.8306506229433}
16  Losses {'ner': 565.1473455251715}
17  Losses {'ner': 541.1816364927284}
18  Losses {'ner': 489.57522128769125}
19  Losses {'ner': 408.41610793219195}
20  Losses {'ner': 541.9054432752819}
21  Losses {'ner': 535.6802834143369}
22  Losses {'ner': 442.6166852567742}
23  Losses {'ner': 584.8445779486016}
24  Losses {'ner': 537.4697011536322}
25  Losses {'ner': 507.19692543824726}
26  Losses {'ner': 542.165

In [15]:
# test the trained model
for text, _ in base_teste:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
    print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])
    print()

Entities [('av. ipiranga loja 149', 'LOGRA')]
Tokens [('av', 'LOGRA', 3), ('.', 'LOGRA', 1), ('ipiranga', 'LOGRA', 1), ('loja', 'LOGRA', 1), ('149', 'LOGRA', 1), ('jardim', '', 2), ('botanico', '', 2), ('90610-000', '', 2), ('porto', '', 2), ('alegre', '', 2), ('rs', '', 2)]

Entities [('avenida presidentegetuilio vargas bela', 'LOGRA')]
Tokens [('avenida', 'LOGRA', 3), ('presidentegetuilio', 'LOGRA', 1), ('vargas', 'LOGRA', 1), ('bela', 'LOGRA', 1), ('vista', '', 2), ('ii', '', 2), ('45996-000', '', 2), ('teixeira', '', 2), ('de', '', 2), ('freitas', '', 2), ('ba', '', 2)]

Entities [('rua f quadra 128 - lote', 'LOGRA')]
Tokens [('rua', 'LOGRA', 3), ('f', 'LOGRA', 1), ('quadra', 'LOGRA', 1), ('128', 'LOGRA', 1), ('-', 'LOGRA', 1), ('lote', 'LOGRA', 1), ('13', '', 2), ('uniao', '', 2), ('68515-000', '', 2), ('parauapebas', '', 2), ('pa', '', 2)]

Entities [('rua brigadeiro franco 2300', 'LOGRA')]
Tokens [('rua', 'LOGRA', 3), ('brigadeiro', 'LOGRA', 1), ('franco', 'LOGRA', 1), ('2300', 

Entities [('pca.dr.henrique sampaio 150', 'LOGRA')]
Tokens [('pca.dr.henrique', 'LOGRA', 3), ('sampaio', 'LOGRA', 1), ('150', 'LOGRA', 1), ('centro', '', 2), ('45745-000', '', 2), ('ibicarai', '', 2), ('ba', '', 2)]

Entities [('av. expedito garcia 75', 'LOGRA')]
Tokens [('av', 'LOGRA', 3), ('.', 'LOGRA', 1), ('expedito', 'LOGRA', 1), ('garcia', 'LOGRA', 1), ('75', 'LOGRA', 1), ('campo', '', 2), ('grande', '', 2), ('29140-000', '', 2), ('cariacica', '', 2), ('es', '', 2)]

Entities [('avenida antonio cunha 700', 'LOGRA')]
Tokens [('avenida', 'LOGRA', 3), ('antonio', 'LOGRA', 1), ('cunha', 'LOGRA', 1), ('700', 'LOGRA', 1), ('centro', '', 2), ('84200-000', '', 2), ('jaguariaiva', '', 2), ('pr', '', 2)]

Entities [('alameda dos maracatins 634/636', 'LOGRA')]
Tokens [('alameda', 'LOGRA', 3), ('dos', 'LOGRA', 1), ('maracatins', 'LOGRA', 1), ('634/636', 'LOGRA', 1), ('indian', '', 2), ('�', '', 2), ('polis', '', 2), ('04089-001', '', 2), ('sao', '', 2), ('paulo', '', 2), ('sp', '', 2)]

Enti

Entities [('rua dona chiquinha de mattos', 'LOGRA')]
Tokens [('rua', 'LOGRA', 3), ('dona', 'LOGRA', 1), ('chiquinha', 'LOGRA', 1), ('de', 'LOGRA', 1), ('mattos', 'LOGRA', 1), ('centro', '', 2), ('12020-010', '', 2), ('taubate', '', 2), ('sp', '', 2)]

Entities [('av. em�lio bosco 201', 'LOGRA')]
Tokens [('av', 'LOGRA', 3), ('.', 'LOGRA', 1), ('em', 'LOGRA', 1), ('�', 'LOGRA', 1), ('lio', 'LOGRA', 1), ('bosco', 'LOGRA', 1), ('201', 'LOGRA', 1), ('condom', '', 2), ('�', '', 2), ('nio', '', 2), ('coronel', '', 2), ('jardim', '', 2), ('morumbi', '', 2), ('13180-000', '', 2), ('sumare', '', 2), ('sp', '', 2)]

Entities [('avenida das na��', 'LOGRA')]
Tokens [('avenida', 'LOGRA', 3), ('das', 'LOGRA', 1), ('na', 'LOGRA', 1), ('�', 'LOGRA', 1), ('�', 'LOGRA', 1), ('es', '', 2), ('unidas', '', 2), ('12559', '', 2), ('brooklin', '', 2), ('paulista', '', 2), ('04578-903', '', 2), ('sao', '', 2), ('paulo', '', 2), ('sp', '', 2)]

Entities [('pca.pref.mario carneiro 23', 'LOGRA')]
Tokens [('pca.pre

Entities [('rua carapinima num 2200', 'LOGRA')]
Tokens [('rua', 'LOGRA', 3), ('carapinima', 'LOGRA', 1), ('num', 'LOGRA', 1), ('2200', 'LOGRA', 1), ('loja', '', 2), ('115a', '', 2), ('benfica', '', 2), ('60015-290', '', 2), ('fortaleza', '', 2), ('ce', '', 2)]

Entities [('r rubens pagani', 'LOGRA')]
Tokens [('r', 'LOGRA', 3), ('rubens', 'LOGRA', 1), ('pagani', 'LOGRA', 1), ('/56', '', 2), ('jd', '', 2), ('estoril', '', 2), ('iv', '', 2), ('17016-210', '', 2), ('bauru', '', 2), ('sp', '', 2)]

Entities [('al. mamore 17.andar', 'LOGRA')]
Tokens [('al', 'LOGRA', 3), ('.', 'LOGRA', 1), ('mamore', 'LOGRA', 1), ('17.andar', 'LOGRA', 1), ('ed.crystal', '', 2), ('tower', '', 2), ('a', '', 2), ('industrial', '', 2), ('06454-040', '', 2), ('barueri', '', 2), ('sp', '', 2)]

Entities [('av. dr ademar de barros 800', 'LOGRA')]
Tokens [('av', 'LOGRA', 3), ('.', 'LOGRA', 1), ('dr', 'LOGRA', 1), ('ademar', 'LOGRA', 1), ('de', 'LOGRA', 1), ('barros', 'LOGRA', 1), ('800', 'LOGRA', 1), ('centro', '', 2

Entities [('rua nhonho do livramento', 'LOGRA')]
Tokens [('rua', 'LOGRA', 3), ('nhonho', 'LOGRA', 1), ('do', 'LOGRA', 1), ('livramento', 'LOGRA', 1), ('centro', '', 2), ('15910-000', '', 2), ('monte', '', 2), ('alto', '', 2), ('sp', '', 2)]

Entities [('rua do catete 355-a', 'LOGRA')]
Tokens [('rua', 'LOGRA', 3), ('do', 'LOGRA', 1), ('catete', 'LOGRA', 1), ('355-a', 'LOGRA', 1), ('e', '', 2), ('357', '', 2), ('catete', '', 2), ('22220-001', '', 2), ('rio', '', 2), ('de', '', 2), ('janeiro', '', 2), ('rj', '', 2)]

Entities [('av. brasil 92', 'LOGRA')]
Tokens [('av', 'LOGRA', 3), ('.', 'LOGRA', 1), ('brasil', 'LOGRA', 1), ('92', 'LOGRA', 1), ('centro', '', 2), ('83850-000', '', 2), ('agudos', '', 2), ('do', '', 2), ('sul', '', 2), ('pr', '', 2)]

Entities [('av.jeronimo de albuquerque s/n', 'LOGRA')]
Tokens [('av.jeronimo', 'LOGRA', 3), ('de', 'LOGRA', 1), ('albuquerque', 'LOGRA', 1), ('s', 'LOGRA', 1), ('/', 'LOGRA', 1), ('n', 'LOGRA', 1), ('.', '', 2), ('cohabanili', '', 2), ('65051-2

...

Entities [('av sao paulo s/', 'LOGRA')]
Tokens [('av', 'LOGRA', 3), ('sao', 'LOGRA', 1), ('paulo', 'LOGRA', 1), ('s', 'LOGRA', 1), ('/', 'LOGRA', 1), ('n', '', 2), ('lt', '', 2), ('13', '', 2), ('q', '', 2), ('35', '', 2), ('a', '', 2), ('vila', '', 2), ('brasilia', '', 2), ('74905-770', '', 2), ('aparecida', '', 2), ('de', '', 2), ('goiania', '', 2), ('go', '', 2)]

Entities [('av.ataulfo de paiva 23', 'LOGRA')]
Tokens [('av.ataulfo', 'LOGRA', 3), ('de', 'LOGRA', 1), ('paiva', 'LOGRA', 1), ('23', 'LOGRA', 1), ('lojas', '', 2), ('a', '', 2), ('/', '', 2), ('b', '', 2), ('leblon', '', 2), ('22440-032', '', 2), ('rio', '', 2), ('de', '', 2), ('janeiro', '', 2), ('rj', '', 2)]

Entities [('avenida antonio abrahao caram 820', 'LOGRA')]
Tokens [('avenida', 'LOGRA', 3), ('antonio', 'LOGRA', 1), ('abrahao', 'LOGRA', 1), ('caram', 'LOGRA', 1), ('820', 'LOGRA', 1), ('lojas', '', 2), ('04', '', 2), ('05', '', 2), ('06', '', 2), ('07', '', 2), ('11', '', 2), ('12', '', 2), ('13', '', 2), ('e', ''

Entities [('r.do comercio 239', 'LOGRA')]
Tokens [('r.do', 'LOGRA', 3), ('comercio', 'LOGRA', 1), ('239', 'LOGRA', 1), ('centro', '', 2), ('23900-565', '', 2), ('angra', '', 2), ('dos', '', 2), ('reis', '', 2), ('rj', '', 2)]

Entities [('r heraclito villar sn', 'LOGRA')]
Tokens [('r', 'LOGRA', 3), ('heraclito', 'LOGRA', 1), ('villar', 'LOGRA', 1), ('sn', 'LOGRA', 1), ('centro', '', 2), ('59570-000', '', 2), ('ceara-mirim', '', 2), ('rn', '', 2)]

Entities [('rua comendador parada 137', 'LOGRA')]
Tokens [('rua', 'LOGRA', 3), ('comendador', 'LOGRA', 1), ('parada', 'LOGRA', 1), ('137', 'LOGRA', 1), ('centro', '', 2), ('18170-000', '', 2), ('piedade', '', 2), ('sp', '', 2)]

Entities [('avenida dos italianos 340', 'LOGRA')]
Tokens [('avenida', 'LOGRA', 3), ('dos', 'LOGRA', 1), ('italianos', 'LOGRA', 1), ('340', 'LOGRA', 1), ('rocha', '', 2), ('miranda', '', 2), ('21510-104', '', 2), ('rio', '', 2), ('de', '', 2), ('janeiro', '', 2), ('rj', '', 2)]

Entities [('rua capit�o frederico teixei

Entities [('q acsv se 22 lote 23 av lo 05 (206', 'LOGRA')]
Tokens [('q', 'LOGRA', 3), ('acsv', 'LOGRA', 1), ('se', 'LOGRA', 1), ('22', 'LOGRA', 1), ('lote', 'LOGRA', 1), ('23', 'LOGRA', 1), ('av', 'LOGRA', 1), ('lo', 'LOGRA', 1), ('05', 'LOGRA', 1), ('(', 'LOGRA', 1), ('206', 'LOGRA', 1), ('s', '', 2), ('-', '', 2), ('a', '', 2), ('sn', '', 2), ('centro', '', 2), ('77020-504', '', 2), ('palmas', '', 2), ('to', '', 2)]

Entities [('avenida cavalheiro paschoal innechi 1460', 'LOGRA')]
Tokens [('avenida', 'LOGRA', 3), ('cavalheiro', 'LOGRA', 1), ('paschoal', 'LOGRA', 1), ('innechi', 'LOGRA', 1), ('1460', 'LOGRA', 1), ('independ', '', 2), ('�', '', 2), ('ncia', '', 2), ('14076-010', '', 2), ('ribeirao', '', 2), ('preto', '', 2), ('sp', '', 2)]

Entities [('rua maria freitas 103 a', 'LOGRA')]
Tokens [('rua', 'LOGRA', 3), ('maria', 'LOGRA', 1), ('freitas', 'LOGRA', 1), ('103', 'LOGRA', 1), ('a', 'LOGRA', 1), ('madureira', '', 2), ('21351-010', '', 2), ('rio', '', 2), ('de', '', 2), ('janeiro

Entities [('rodovia amaral peixoto 5251', 'LOGRA')]
Tokens [('rodovia', 'LOGRA', 3), ('amaral', 'LOGRA', 1), ('peixoto', 'LOGRA', 1), ('5251', 'LOGRA', 1), ('novo', '', 2), ('rio', '', 2), ('das', '', 2), ('ostras', '', 2), ('28890-000', '', 2), ('rio', '', 2), ('das', '', 2), ('ostras', '', 2), ('rj', '', 2)]

Entities [('av. ernani do amaral peixoto 363', 'LOGRA')]
Tokens [('av', 'LOGRA', 3), ('.', 'LOGRA', 1), ('ernani', 'LOGRA', 1), ('do', 'LOGRA', 1), ('amaral', 'LOGRA', 1), ('peixoto', 'LOGRA', 1), ('363', 'LOGRA', 1), ('centro', '', 2), ('24020-072', '', 2), ('niteroi', '', 2), ('rj', '', 2)]

Entities [('rua engenheiro antunes 222', 'LOGRA')]
Tokens [('rua', 'LOGRA', 3), ('engenheiro', 'LOGRA', 1), ('antunes', 'LOGRA', 1), ('222', 'LOGRA', 1), ('360305', '', 2), ('centro', '', 2), ('39800-019', '', 2), ('teofilo', '', 2), ('otoni', '', 2), ('mg', '', 2)]

Entities [('rua julio de castilhos 370', 'LOGRA')]
Tokens [('rua', 'LOGRA', 3), ('julio', 'LOGRA', 1), ('de', 'LOGRA', 1), (

Entities [('av mister hull padre andrade', 'LOGRA')]
Tokens [('av', 'LOGRA', 3), ('mister', 'LOGRA', 1), ('hull', 'LOGRA', 1), ('padre', 'LOGRA', 1), ('andrade', 'LOGRA', 1), ('antonio', '', 2), ('bezerra', '', 2), ('60356-412', '', 2), ('fortaleza', '', 2), ('ce', '', 2)]

Entities [('avenida ministro geraldo barreto sobral', 'LOGRA')]
Tokens [('avenida', 'LOGRA', 3), ('ministro', 'LOGRA', 1), ('geraldo', 'LOGRA', 1), ('barreto', 'LOGRA', 1), ('sobral', 'LOGRA', 1), ('jardins', '', 2), ('49026-010', '', 2), ('aracaju', '', 2), ('se', '', 2)]

Entities [('rua olavo barreto viana', 'LOGRA')]
Tokens [('rua', 'LOGRA', 3), ('olavo', 'LOGRA', 1), ('barreto', 'LOGRA', 1), ('viana', 'LOGRA', 1), ('moinhos', '', 2), ('de', '', 2), ('vento', '', 2), ('90570-070', '', 2), ('porto', '', 2), ('alegre', '', 2), ('rs', '', 2)]

Entities [('av julio borella 1085', 'LOGRA')]
Tokens [('av', 'LOGRA', 3), ('julio', 'LOGRA', 1), ('borella', 'LOGRA', 1), ('1085', 'LOGRA', 1), ('centro', '', 2), ('99150-000

In [20]:
phrase = "QNE 20 12  teste de endereço pedro leonardo"

doc = nlp(phrase)
print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Entities [('QNE 20 12  ', 'LOGRA')]
Tokens [('QNE', 'LOGRA', 3), ('20', 'LOGRA', 1), ('12', 'LOGRA', 1), (' ', 'LOGRA', 1), ('teste', '', 2), ('de', '', 2), ('endereço', '', 2), ('pedro', '', 2), ('leonardo', '', 2)]


In [ ]:
from spacy.gold import GoldParse
from spacy.scorer import Scorer

def evaluate(ner_model, examples):
    scorer = Scorer()
    for input_, annot in examples:
        doc_gold_text = ner_model.make_doc(input_)
        gold = GoldParse(doc_gold_text, entities=annot)
        pred_value = ner_model(input_)
        scorer.score(pred_value, gold)
    return scorer.scores

In [ ]:
# save model to output directory
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

In [ ]:
loaded_model = spacy.load(output_dir)
evaluate(loaded_model, base_teste_final)